In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline

In [ ]:
results_path = os.path.join(os.environ['HOME'], 'fits/neurobiases/tc_sweep.npz')

In [ ]:
results = np.load(results_path)

In [ ]:
a_trues = results['a_trues']
b_trues = results['b_trues']
a_hats = results['a_hats']
b_hats = results['b_hats']

In [ ]:
n_nz_a = np.unique(np.count_nonzero(a_trues, axis=-1)).item()
n_nz_b = np.unique(np.count_nonzero(b_trues, axis=-1)).item()
n_means, n_models, n_datasets = a_hats.shape[:-1]

In [ ]:
a_trues_nz = np.zeros((n_means, n_models, n_nz_a))
b_trues_nz = np.zeros((n_means, n_models, n_nz_b))
a_bias = np.zeros((n_means, n_models, n_datasets, n_nz_a))
b_bias = np.zeros((n_means, n_models, n_datasets, n_nz_b))
a_bias_norm = np.zeros_like(a_bias)
b_bias_norm = np.zeros_like(b_bias)

In [ ]:
for mean_idx in range(n_means):
    for model_idx in range(n_models):
        current_a = a_trues[mean_idx, model_idx]
        current_b = b_trues[mean_idx, model_idx]
        a_trues_nz[mean_idx, model_idx] = current_a[current_a != 0]
        b_trues_nz[mean_idx, model_idx] = current_b[current_b != 0]
        for dataset_idx in range(n_datasets):
            a_bias_temp = \
                a_hats[mean_idx, model_idx, dataset_idx][current_a != 0] - a_trues_nz[mean_idx, model_idx]
            b_bias_temp = \
                b_hats[mean_idx, model_idx, dataset_idx][current_b != 0] - b_trues_nz[mean_idx, model_idx]
            a_bias[mean_idx, model_idx, dataset_idx] = a_bias_temp
            a_bias_norm[mean_idx, model_idx, dataset_idx] = a_bias_temp / np.abs(a_trues_nz[mean_idx, model_idx])
            b_bias[mean_idx, model_idx, dataset_idx] = b_bias_temp
            b_bias_norm[mean_idx, model_idx, dataset_idx] = b_bias_temp / np.abs(b_trues_nz[mean_idx, model_idx])

In [ ]:
a_sums = np.sum(a_trues_nz, axis=-1)
means = np.linspace(-3, 3, 30)

In [ ]:
a_bias_med = np.median(np.mean(a_bias, axis=3), axis=2)
b_bias_med = np.median(np.mean(b_bias, axis=3), axis=2)
a_bias_avg = np.mean(a_bias_norm, axis=(2, 3))
b_bias_avg = np.mean(b_bias_norm, axis=(2, 3))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)

axes[0].scatter(a_sums.ravel(), a_bias_med.ravel(),
                alpha=0.5,
                color='black')
axes[1].scatter(a_sums.ravel(), b_bias_med.ravel(),
                alpha=0.5,
                color='black')

for ax in axes:
    ax.set_xlim([-20, 20])
    ax.set_xlabel(r'\textbf{Coupling Sum}', fontsize=16)
    ax.tick_params(labelsize=14)
    ax.axvline(0, color='red', linestyle='--')

axes[0].set_ylim(bottom=0)
axes[0].set_ylabel(r'\textbf{Median Coupling Bias}', fontsize=16)
axes[1].set_ylim(top=0)
axes[1].set_ylabel(r'\textbf{Median Tuning Bias}', fontsize=16)

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)

axes[0].scatter(a_sums.ravel(), a_bias_avg.ravel(),
                alpha=0.5,
                color='black')
axes[1].scatter(a_sums.ravel(), b_bias_avg.ravel(),
                alpha=0.5,
                color='black')

for ax in axes:
    ax.set_xlim([-20, 20])
    ax.set_xlabel(r'\textbf{Coupling Sum}', fontsize=16)
    ax.tick_params(labelsize=14)
    ax.axvline(0, color='red', linestyle='--')

axes[0].set_ylim(bottom=0, top=2)
axes[0].set_ylabel(r'\textbf{Mean Coupling Bias}', fontsize=16)
axes[1].set_ylim(top=0)
axes[1].set_ylabel(r'\textbf{Mean Tuning Bias}', fontsize=16)

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)

for model in range(n_models):
    axes[0].scatter(means, a_bias_med[:, model],
                    alpha=0.5,
                    color='black')
    axes[1].scatter(means, b_bias_med[:, model],
                    alpha=0.5,
                    color='black')

for ax in axes:
    ax.set_xlim([-3, 3])
    ax.set_xlabel(r'\textbf{Mean of Coupling Distribution}', fontsize=16)
    ax.tick_params(labelsize=14)
    ax.axvline(0, color='red', linestyle='--')

axes[0].set_ylim(bottom=0)
axes[0].set_ylabel(r'\textbf{Median Coupling Bias}', fontsize=16)
axes[1].set_ylim(top=0)
axes[1].set_ylabel(r'\textbf{Median Tuning Bias}', fontsize=16)

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)

for model in range(n_models):
    axes[0].scatter(means, a_bias_avg[:, model],
                    alpha=0.5,
                    color='black')
    axes[1].scatter(means, b_bias_avg[:, model],
                    alpha=0.5,
                    color='black')

for ax in axes:
    ax.set_xlim([-3, 3])
    ax.set_xlabel(r'\textbf{Coupling Sum}', fontsize=16)
    ax.tick_params(labelsize=14)
    ax.axvline(0, color='red', linestyle='--')

axes[0].set_ylim(bottom=0, top=2)
axes[0].set_ylabel(r'\textbf{Mean of Coupling Distribution}', fontsize=16)
axes[1].set_ylim(top=0)
axes[1].set_ylabel(r'\textbf{Median Tuning Bias}', fontsize=16)

plt.tight_layout()